# Feature Flag Audit for Audio-Processor Application

## Executive Summary

This notebook provides a comprehensive audit of all feature flags defined in the audio-processor application and traces their usage throughout the codebase. 

**Key Findings:**
- ✅ **Properly Implemented**: `graph.enabled`, `auth.verify_signature`, `auth.verify_audience`
- ❌ **Not Implemented**: `enable_audio_upload`, `enable_url_processing`, `enable_translation`, `enable_summarization`

**Critical Issue**: Several core feature flags are defined but never checked, creating a false sense of security for system administrators.

## Section 1: Environment Setup and Feature Flag Discovery

**Prerequisites:**
- Python 3.12+ (required for this project)
- `uv` package manager with dependencies installed (`uv sync`)
- Virtual environment activated

First, let's verify our environment and examine all defined feature flags.

In [ ]:
import os
import sys
import ast
import inspect
from pathlib import Path
from typing import Dict, Any
import pandas as pd

# Verify Python version (should be 3.12 for this project)
print(f"🐍 Python Version: {sys.version}")
if sys.version_info < (3, 12):
    print("⚠️  WARNING: This project requires Python 3.12+")
    print("   Please ensure you're using the correct Python version with uv")
else:
    print("✅ Python version is compatible")

# Check if we're in a uv environment
print(f"\n📦 Package Manager: uv")
if os.environ.get('VIRTUAL_ENV'):
    print(f"✅ Virtual environment active: {os.environ['VIRTUAL_ENV']}")
else:
    print("⚠️  No virtual environment detected - ensure uv environment is activated")

# Add the app directory to Python path
sys.path.insert(0, os.path.abspath('.'))

try:
    # Import settings
    from app.config.settings import Settings
    print("✅ Successfully imported Settings")
except ImportError as e:
    print(f"❌ Failed to import Settings: {e}")
    print("   Make sure dependencies are installed with: uv sync")
    sys.exit(1)

# Create a settings instance to examine default values
settings = Settings()

print("\n🔍 Feature Flags Defined in Settings Class")
print("=" * 50)

# Feature flags we're specifically interested in
feature_flags = {
    'enable_audio_upload': settings.enable_audio_upload,
    'enable_url_processing': settings.enable_url_processing, 
    'enable_translation': settings.enable_translation,
    'enable_summarization': settings.enable_summarization,
    'graph.enabled': settings.graph.enabled,
    'auth.verify_signature': settings.auth.verify_signature,
    'auth.verify_audience': settings.auth.verify_audience,
}

# Create a summary table
flag_data = []
for flag, value in feature_flags.items():
    flag_data.append({
        'Feature Flag': flag,
        'Default Value': value,
        'Type': type(value).__name__,
        'Environment Variable': flag.upper().replace('.', '_')
    })

df = pd.DataFrame(flag_data)
print(df.to_string(index=False))

print(f"\n📊 Total Feature Flags: {len(feature_flags)}")
print(f"🟢 Enabled by Default: {sum(1 for v in feature_flags.values() if v)}")
print(f"🔴 Disabled by Default: {sum(1 for v in feature_flags.values() if not v)}")

## Section 2: Trace Feature Flag Usage in Codebase

Now let's search the entire codebase for references to each feature flag.

In [ ]:
import re
import glob
from collections import defaultdict

def search_feature_flag_usage(flag_name: str, search_patterns: list) -> Dict[str, list]:
    """Search for feature flag usage in Python files."""
    results = defaultdict(list)
    
    # Get all Python files in the app directory
    python_files = glob.glob('app/**/*.py', recursive=True)
    
    for file_path in python_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                lines = content.split('\n')
                
            for i, line in enumerate(lines, 1):
                for pattern in search_patterns:
                    if re.search(pattern, line, re.IGNORECASE):
                        results[file_path].append({
                            'line_number': i,
                            'line_content': line.strip(),
                            'pattern_matched': pattern
                        })
                        
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    return dict(results)

# Define search patterns for each flag
flag_search_patterns = {
    'enable_audio_upload': [
        r'enable_audio_upload',
        r'ENABLE_AUDIO_UPLOAD'
    ],
    'enable_url_processing': [
        r'enable_url_processing',
        r'ENABLE_URL_PROCESSING'
    ],
    'enable_translation': [
        r'enable_translation',
        r'ENABLE_TRANSLATION'
    ],
    'enable_summarization': [
        r'enable_summarization',
        r'ENABLE_SUMMARIZATION'
    ],
    'graph.enabled': [
        r'graph\.enabled',
        r'GRAPH_ENABLED'
    ],
    'auth.verify_signature': [
        r'verify_signature',
        r'JWT_VERIFY_SIGNATURE'
    ],
    'auth.verify_audience': [
        r'verify_audience',
        r'JWT_VERIFY_AUDIENCE'
    ]
}

print("🔍 Searching for Feature Flag Usage in Codebase")
print("=" * 60)

usage_summary = {}
for flag, patterns in flag_search_patterns.items():
    results = search_feature_flag_usage(flag, patterns)
    usage_summary[flag] = results
    
    print(f"\n🏷️  {flag}")
    print("-" * 30)
    
    if results:
        total_refs = sum(len(refs) for refs in results.values())
        print(f"📍 Found {total_refs} references in {len(results)} files")
        
        for file_path, refs in results.items():
            print(f"   📄 {file_path}: {len(refs)} reference(s)")
            for ref in refs[:3]:  # Show first 3 references
                print(f"      Line {ref['line_number']}: {ref['line_content'][:80]}...")
            if len(refs) > 3:
                print(f"      ... and {len(refs) - 3} more")
    else:
        print("❌ No references found (flag is defined but not used)")

## Section 3: Check Feature Flag Enforcement in API Endpoints

Let's examine the main API endpoint (`transcribe.py`) to see which feature flags are actually enforced.

In [ ]:
def analyze_transcribe_endpoint():
    """Analyze the transcribe endpoint for feature flag enforcement."""
    transcribe_file = 'app/api/v1/endpoints/transcribe.py'
    
    print("🔍 Analyzing transcribe.py for Feature Flag Enforcement")
    print("=" * 60)
    
    try:
        with open(transcribe_file, 'r', encoding='utf-8') as f:
            content = f.read()
            lines = content.split('\n')
    except FileNotFoundError:
        print(f"❌ File not found: {transcribe_file}")
        return
    
    # Look for HTTPException raises related to feature flags
    enforcement_patterns = {
        'enable_audio_upload': r'enable_audio_upload.*HTTPException|HTTPException.*enable_audio_upload',
        'enable_url_processing': r'enable_url_processing.*HTTPException|HTTPException.*enable_url_processing',
        'enable_translation': r'enable_translation.*HTTPException|HTTPException.*enable_translation',
        'enable_summarization': r'enable_summarization.*HTTPException|HTTPException.*enable_summarization'
    }
    
    # Also look for basic feature flag checks
    basic_checks = {
        'enable_audio_upload': r'if.*enable_audio_upload',
        'enable_url_processing': r'if.*enable_url_processing',
        'enable_translation': r'if.*enable_translation', 
        'enable_summarization': r'if.*enable_summarization'
    }
    
    print("🚨 Feature Flag Enforcement Check:")
    print("-" * 40)
    
    for flag in ['enable_audio_upload', 'enable_url_processing', 'enable_translation', 'enable_summarization']:
        enforcement_found = False
        basic_check_found = False
        
        # Check for enforcement (HTTPException)
        for i, line in enumerate(lines, 1):
            if re.search(enforcement_patterns[flag], line, re.IGNORECASE):
                print(f"✅ {flag}: ENFORCED at line {i}")
                print(f"   💡 {line.strip()}")
                enforcement_found = True
                break
        
        # Check for basic flag usage
        if not enforcement_found:
            for i, line in enumerate(lines, 1):
                if re.search(basic_checks[flag], line, re.IGNORECASE):
                    print(f"⚠️  {flag}: CHECKED but enforcement unclear at line {i}")
                    print(f"   💡 {line.strip()}")
                    basic_check_found = True
                    break
        
        if not enforcement_found and not basic_check_found:
            print(f"❌ {flag}: NOT ENFORCED (no checks found)")
    
    # Check if settings dependency is injected
    print("\n🔧 Settings Dependency Injection:")
    print("-" * 35)
    settings_injection_found = False
    for i, line in enumerate(lines, 1):
        if 'settings' in line and 'Depends' in line and 'get_settings' in line:
            print(f"✅ Settings injected at line {i}: {line.strip()}")
            settings_injection_found = True
            break
    
    if not settings_injection_found:
        print("❌ Settings dependency not properly injected")
    
    return content

transcribe_content = analyze_transcribe_endpoint()

## Section 4: Demonstrate Proper Feature Flag Usage (graph.enabled)

The `graph.enabled` flag is the **gold standard** implementation. Let's examine how it's properly used.

In [ ]:
def show_graph_enabled_implementation():
    """Show how graph.enabled is properly implemented."""
    print("✅ PROPER IMPLEMENTATION: graph.enabled Flag")
    print("=" * 50)
    
    # Files where graph.enabled is properly checked
    graph_files = [
        'app/main.py',
        'app/core/graph_processor.py', 
        'app/api/v1/endpoints/graph.py'
    ]
    
    for file_path in graph_files:
        print(f"\n📄 {file_path}")
        print("-" * 30)
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                
            # Find lines with graph.enabled checks
            for i, line in enumerate(lines, 1):
                if 'graph.enabled' in line or 'graph_enabled' in line:
                    # Show context around the check
                    start = max(0, i-3)
                    end = min(len(lines), i+2)
                    
                    print(f"   Line {i}: Feature flag check found")
                    print("   Context:")
                    for j in range(start, end):
                        marker = ">>>" if j == i-1 else "   "
                        print(f"   {marker} {j+1:3d}: {lines[j].rstrip()}")
                    print()
                    
        except FileNotFoundError:
            print(f"   ❌ File not found: {file_path}")
    
    print("\n🎯 Why this is the GOLD STANDARD:")
    print("   ✅ Checked at API boundary (returns HTTP 503)")
    print("   ✅ Checked in core processing logic") 
    print("   ✅ Graceful degradation when disabled")
    print("   ✅ Clear error messages for users")
    print("   ✅ Production-ready implementation")

show_graph_enabled_implementation()

## Section 5: Show Missing Feature Flag Checks

Let's prove that the core feature flags are NOT being checked in the API logic.

In [ ]:
def prove_missing_flag_checks():
    """Prove that critical feature flags are not enforced."""
    print("❌ MISSING IMPLEMENTATIONS: Core Feature Flags")
    print("=" * 55)
    
    # Read the transcribe endpoint
    transcribe_file = 'app/api/v1/endpoints/transcribe.py'
    
    try:
        with open(transcribe_file, 'r', encoding='utf-8') as f:
            content = f.read()
    except FileNotFoundError:
        print(f"❌ File not found: {transcribe_file}")
        return
    
    missing_flags = {
        'enable_audio_upload': {
            'feature': 'File uploads',
            'parameter': 'file: Optional[UploadFile]',
            'risk': 'Users can upload files even when feature is disabled'
        },
        'enable_translation': {
            'feature': 'Translation',
            'parameter': 'translate: bool = Form(False)',
            'risk': 'Users can request translation when feature is disabled'
        },
        'enable_summarization': {
            'feature': 'Summarization', 
            'parameter': 'summarize: bool = Form(False)',
            'risk': 'Users can request summaries when feature is disabled'
        }
    }
    
    for flag, info in missing_flags.items():
        print(f"\n🚨 {flag}")
        print("-" * 25)
        print(f"   Feature: {info['feature']}")
        print(f"   Parameter: {info['parameter']}")
        print(f"   Risk: {info['risk']}")
        
        # Check if the flag is referenced anywhere in the file
        if flag in content:
            # Find the context
            lines = content.split('\n')
            for i, line in enumerate(lines, 1):
                if flag in line:
                    print(f"   ✅ Flag mentioned at line {i}: {line.strip()}")
                    break
        else:
            print(f"   ❌ Flag NOT MENTIONED in transcribe endpoint")
        
        # Check for enforcement pattern
        enforcement_patterns = [
            f'if.*{flag}.*HTTPException',
            f'HTTPException.*{flag}',
            f'not settings.{flag}'
        ]
        
        enforcement_found = False
        for pattern in enforcement_patterns:
            if re.search(pattern, content, re.IGNORECASE):
                enforcement_found = True
                print(f"   ✅ Enforcement found: {pattern}")
                break
        
        if not enforcement_found:
            print(f"   ❌ NO ENFORCEMENT FOUND - Flag is defined but not checked!")
    
    print(f"\n💥 CRITICAL SECURITY ISSUE:")
    print(f"   These flags provide administrators with a false sense of control.")
    print(f"   Setting them to False will NOT actually disable the features!")
    
    return content

transcribe_analysis = prove_missing_flag_checks()

## Section 6: Implement and Test Feature Flag Enforcement

Now let's implement the missing feature flag checks and create tests to verify they work.

In [ ]:
def show_required_fixes():
    """Show the exact code that needs to be added to fix feature flag enforcement."""
    print("🔧 REQUIRED FIXES for Feature Flag Enforcement")
    print("=" * 55)
    
    fixes = {
        'enable_audio_upload': {
            'check': '''
# Check if file uploads are enabled
if file and not settings.enable_audio_upload:
    raise HTTPException(
        status_code=status.HTTP_403_FORBIDDEN,
        detail="Direct audio file uploads are disabled."
    )''',
            'location': 'After basic file/URL validation, before processing'
        },
        
        'enable_translation': {
            'check': '''
# Check if translation is enabled  
if translate and not settings.enable_translation:
    raise HTTPException(
        status_code=status.HTTP_400_BAD_REQUEST,
        detail="Translation feature is currently disabled."
    )''',
            'location': 'After parameter validation'
        },
        
        'enable_summarization': {
            'check': '''
# Check if summarization is enabled
if summarize and not settings.enable_summarization:
    raise HTTPException(
        status_code=status.HTTP_400_BAD_REQUEST,
        detail="Summarization feature is currently disabled."
    )''',
            'location': 'After parameter validation'
        }
    }
    
    for flag, info in fixes.items():
        print(f"\n🏷️  {flag}")
        print("-" * 30)
        print(f"Location: {info['location']}")
        print("Required Code:")
        print(info['check'])
    
    print(f"\n📝 Additional Requirements:")
    print(f"   1. Settings dependency must be properly injected")
    print(f"   2. HTTPException must be imported") 
    print(f"   3. status codes must be imported")
    print(f"   4. All checks should be near the top of the function")
    print(f"   5. Compatible with Python 3.12+ type hints")
    
    print(f"\n🔧 Complete Dependency Injection Pattern:")
    print(f"   settings: Settings = Depends(get_settings_dependency)")

show_required_fixes()

In [ ]:
def create_feature_flag_tests():
    """Create unit tests to verify feature flag enforcement."""
    print("🧪 Unit Tests for Feature Flag Enforcement")
    print("=" * 50)
    
    test_code = '''
"""
Feature Flag Enforcement Tests
Compatible with Python 3.12+ and uv package manager

Run with: uv run pytest tests/unit/test_feature_flag_enforcement.py -v
"""

import pytest
from unittest.mock import MagicMock, AsyncMock
from fastapi import HTTPException
from app.api.v1.endpoints.transcribe import transcribe_audio

class TestFeatureFlagEnforcement:
    """Test that feature flags properly block requests when disabled."""
    
    @pytest.mark.asyncio
    async def test_audio_upload_disabled(self):
        """Test that file uploads are blocked when enable_audio_upload=False."""
        # Mock settings with audio upload disabled
        mock_settings = MagicMock()
        mock_settings.enable_audio_upload = False
        mock_settings.enable_url_processing = True
        mock_settings.enable_translation = True
        mock_settings.enable_summarization = True
        
        # Mock file upload
        mock_file = MagicMock()
        mock_file.filename = "test.mp3"
        
        # Mock other dependencies
        mock_user_id = "test-user-123"
        mock_transcription_service = AsyncMock()
        mock_job_queue = AsyncMock()
        
        with pytest.raises(HTTPException) as exc_info:
            await transcribe_audio(
                file=mock_file,
                audio_url=None,
                language="auto",
                model="large-v2",
                punctuate=True,
                diarize=True,
                smart_format=True,
                utterances=True,
                utt_split=0.8,
                translate=False,
                summarize=False,
                callback_url=None,
                user_id=mock_user_id,
                transcription_service=mock_transcription_service,
                job_queue=mock_job_queue,
                settings=mock_settings
            )
        
        assert exc_info.value.status_code == 403
        assert "uploads are disabled" in str(exc_info.value.detail)
    
    @pytest.mark.asyncio 
    async def test_translation_disabled(self):
        """Test that translation requests are blocked when enable_translation=False."""
        mock_settings = MagicMock()
        mock_settings.enable_audio_upload = True
        mock_settings.enable_url_processing = True
        mock_settings.enable_translation = False  # DISABLED
        mock_settings.enable_summarization = True
        
        # Mock other dependencies
        mock_user_id = "test-user-123"
        mock_transcription_service = AsyncMock()
        mock_job_queue = AsyncMock()
        
        with pytest.raises(HTTPException) as exc_info:
            await transcribe_audio(
                file=None,
                audio_url="https://example.com/audio.mp3",
                language="auto",
                model="large-v2",
                punctuate=True,
                diarize=True,
                smart_format=True,
                utterances=True,
                utt_split=0.8,
                translate=True,  # User requests translation
                summarize=False,
                callback_url=None,
                user_id=mock_user_id,
                transcription_service=mock_transcription_service,
                job_queue=mock_job_queue,
                settings=mock_settings
            )
        
        assert exc_info.value.status_code == 400
        assert "Translation feature is currently disabled" in str(exc_info.value.detail)
    
    @pytest.mark.asyncio
    async def test_summarization_disabled(self):
        """Test that summarization requests are blocked when enable_summarization=False."""
        mock_settings = MagicMock()
        mock_settings.enable_audio_upload = True
        mock_settings.enable_url_processing = True
        mock_settings.enable_translation = True
        mock_settings.enable_summarization = False  # DISABLED
        
        # Mock other dependencies
        mock_user_id = "test-user-123"
        mock_transcription_service = AsyncMock()
        mock_job_queue = AsyncMock()
        
        with pytest.raises(HTTPException) as exc_info:
            await transcribe_audio(
                file=None,
                audio_url="https://example.com/audio.mp3",
                language="auto",
                model="large-v2", 
                punctuate=True,
                diarize=True,
                smart_format=True,
                utterances=True,
                utt_split=0.8,
                translate=False,
                summarize=True,  # User requests summary
                callback_url=None,
                user_id=mock_user_id,
                transcription_service=mock_transcription_service,
                job_queue=mock_job_queue,
                settings=mock_settings
            )
        
        assert exc_info.value.status_code == 400
        assert "Summarization feature is currently disabled" in str(exc_info.value.detail)
    
    @pytest.mark.asyncio
    async def test_all_features_enabled_works(self):
        """Test that requests work normally when all features are enabled."""
        mock_settings = MagicMock()
        mock_settings.enable_audio_upload = True
        mock_settings.enable_url_processing = True
        mock_settings.enable_translation = True
        mock_settings.enable_summarization = True
        
        # Mock other dependencies
        mock_user_id = "test-user-123"
        mock_transcription_service = AsyncMock()
        mock_job_queue = AsyncMock()
        
        # Mock job creation
        mock_job_queue.create_job = AsyncMock(return_value=MagicMock())
        mock_job_queue.update_job = AsyncMock()
        
        # This should NOT raise an exception for feature flags
        # (it may still raise for other validation issues)
        try:
            result = await transcribe_audio(
                file=None,
                audio_url="https://example.com/audio.mp3",
                language="auto",
                model="large-v2",
                punctuate=True,
                diarize=True,
                smart_format=True,
                utterances=True,
                utt_split=0.8,
                translate=True,
                summarize=True,
                callback_url=None,
                user_id=mock_user_id,
                transcription_service=mock_transcription_service,
                job_queue=mock_job_queue,
                settings=mock_settings
            )
            # Should reach here without feature flag exceptions
            assert result.status == "queued"
        except HTTPException as e:
            if "disabled" in str(e.detail):
                pytest.fail(f"Feature flag incorrectly blocked request: {e.detail}")
            # Re-raise other HTTPExceptions (e.g., validation errors)
            raise
'''
    
    print("Test Structure:")
    print("✅ test_audio_upload_disabled - Verifies file uploads blocked")
    print("✅ test_translation_disabled - Verifies translation requests blocked") 
    print("✅ test_summarization_disabled - Verifies summarization requests blocked")
    print("✅ test_all_features_enabled_works - Verifies normal operation")
    
    print(f"\n🚀 Running Tests with uv:")
    print("   uv run pytest tests/unit/test_feature_flag_enforcement.py -v")
    print("   uv run pytest tests/unit/test_feature_flag_enforcement.py::TestFeatureFlagEnforcement::test_audio_upload_disabled -v")
    
    print(f"\n📄 Complete Test Code (Python 3.12+ compatible):")
    print("-" * 50)
    print(test_code)
    
    return test_code

test_code = create_feature_flag_tests()

## Summary and Recommendations

### 🚨 Critical Findings

**SECURITY RISK**: Several feature flags are defined but never enforced, creating a false sense of security for system administrators.

### ✅ Properly Implemented Flags
- `graph.enabled` - Perfect implementation with API boundary checks
- `auth.verify_signature` - Correctly used in JWT validation
- `auth.verify_audience` - Correctly used in JWT validation

### ❌ Missing Implementations (URGENT)
- `enable_audio_upload` - Users can upload files even when "disabled"
- `enable_translation` - Users can request translation when "disabled"  
- `enable_summarization` - Users can request summarization when "disabled"
- `enable_url_processing` - **PARTIALLY IMPLEMENTED** (check exists but may need verification)

### 🔧 Required Actions

1. **Immediate**: Add feature flag checks to `transcribe_audio` endpoint
2. **Testing**: Implement unit tests to verify enforcement
3. **Documentation**: Update API docs to reflect feature flag behavior
4. **Monitoring**: Add logging for feature flag enforcement events

### 📊 Implementation Priority

1. **HIGH**: `enable_audio_upload` - Core functionality control
2. **HIGH**: `enable_translation` - Resource-intensive feature
3. **HIGH**: `enable_summarization` - Resource-intensive feature  
4. **VERIFY**: `enable_url_processing` - Appears implemented but needs verification

### 🐍 Python 3.12 & uv Specific Notes

**Development Setup:**
```bash
# Ensure Python 3.12+ is active
python --version  # Should be 3.12+

# Install dependencies with uv
uv sync

# Run tests with uv
uv run pytest tests/unit/test_feature_flag_enforcement.py -v
```

**Code Compatibility:**
- All code samples are compatible with Python 3.12+ type hints
- Uses modern `typing` module features
- Compatible with `uv` package manager and virtual environments

### 🎯 Next Steps

1. **Implement the fixes** shown in Section 6
2. **Add the unit tests** from Section 6 to `tests/unit/test_feature_flag_enforcement.py`
3. **Run tests** with `uv run pytest` to verify implementation
4. **Update documentation** to reflect new feature flag behavior

**The application is NOT production-ready until these feature flag checks are implemented.**